In [58]:

!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytorch
    - pytorch-cuda=11.8
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    -------------------

In [1]:
!conda install -c huggingface transformers datasets tokenizers evaluate huggingface_hub -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [2]:
!sudo apt-get update
!sudo apt-get install libssl-dev

Hit:1 https://dl.yarnpkg.com/debian stable InRelease
Hit:2 http://deb.debian.org/debian bullseye InRelease
Hit:3 http://deb.debian.org/debian-security bullseye-security InRelease
Hit:4 http://deb.debian.org/debian bullseye-updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libssl-dev is already the newest version (1.1.1n-0+deb11u4).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [5]:
!conda update tokenizers -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - tokenizers


The following packages will be SUPERSEDED by a higher-priority channel:

  tokenizers         huggingface::tokenizers-0.13.0.dev0-p~ --> pkgs/main::tokenizers-0.11.4-py39h3dcd8bd_1 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done


In [22]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline
from datasets import Dataset, ClassLabel, Features, Value
import evaluate
import pandas as pd
import numpy as np

In [61]:
from sklearn.metrics import (
    confusion_matrix,
    cohen_kappa_score,
    classification_report,
    accuracy_score,
    roc_auc_score,
)

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")
data.columns = ['id', 'text', 'label']
data.label = data.label.map({'normal':0, 'incivilidad': 1, 'odio':2})
data = data[['text','label']]
data

,text,label
0,ultimo choro se 2018 que delicia,0
1,Pero es una realidad para muchas mujeres en Ve...,0
2,MALDITA SEAS COMUNA DE ÑUÑOA https://t.co/yN4E...,1
3,Las tontas de #PautaLibre con el tremendo 🌶🌶 ...,1
4,@user @user @user @user @user Devuelvete y and...,2
...,...,...
12209,🔵🔴 La “U” despidió a Dudamel con un empate.\n\...,0
12210,"Armen la casa de puta, loco. Cómanse a Roldán!...",1
12211,@user A mí me penalizaron por merequetengue e ...,0
12212,Me importa un pico si del Pino juega bien o ma...,1


In [5]:
features = Features({"text": Value("string"), "label": ClassLabel(num_classes=3, names=[0,1,2])})

In [6]:
dataset = Dataset.from_pandas(data, features=features).train_test_split(test_size=0.3)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8549
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3665
    })
})

In [8]:
def preprocess_function(examples):

    return tokenizer(examples["text"], truncation=True)

In [9]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8549 [00:00<?, ? examples/s]

Map:   0%|          | 0/3665 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
accuracy = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
id2label = {0: "normal", 1: "incivilidad", 2:'odio'}

label2id = {"normal": 0, "incivilidad": 1, 'odio':3}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(

    "dccuchile/bert-base-spanish-wwm-cased", num_labels=3, id2label=id2label, label2id=label2id

)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [15]:
training_args = TrainingArguments(

    output_dir="hater_model",

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    num_train_epochs=2,

    weight_decay=0.01,

    evaluation_strategy="epoch",

    save_strategy="epoch",

    load_best_model_at_end=True,

)

In [16]:
trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_dataset["train"],

    eval_dataset=tokenized_dataset["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

    compute_metrics=compute_metrics,

)

In [17]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.627700,0.450751,0.817462
2,0.338900,0.463539,0.828104


TrainOutput(global_step=1070, training_loss=0.47175652156366366, metrics={'train_runtime': 330.744, 'train_samples_per_second': 51.696, 'train_steps_per_second': 3.235, 'total_flos': 845415272550492.0, 'train_loss': 0.47175652156366366, 'epoch': 2.0})

In [29]:
device = torch.device('cuda')

In [30]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [33]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

In [45]:
model.push_to_hub('hater_model')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fvillena/hater_model/commit/4133603f913b6f15b15dc92060374d30a39dfd7b', commit_message='Upload BertForSequenceClassification', commit_description='', oid='4133603f913b6f15b15dc92060374d30a39dfd7b', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
tokenizer.push_to_hub('hater_model')

CommitInfo(commit_url='https://huggingface.co/fvillena/hater_model/commit/d30f60be4a40f323843fc04b0caf89f72da84b48', commit_message='Upload tokenizer', commit_description='', oid='d30f60be4a40f323843fc04b0caf89f72da84b48', pr_url=None, pr_revision=None, pr_num=None)

In [52]:
predicted = pipe(dataset['test']['text'])

In [56]:
predicted_probabilities = np.array([[p[0]['score'],p[1]['score'],p[2]['score']] for p in predicted])

In [57]:
predicted_probabilities

array([[0.04451904, 0.93562138, 0.01985958],
       [0.92305464, 0.01400349, 0.06294189],
       [0.01748722, 0.97010887, 0.01240395],
       ...,
       [0.08106995, 0.42701054, 0.49191946],
       [0.00552912, 0.98276937, 0.0117015 ],
       [0.00507274, 0.97307903, 0.02184826]])

In [59]:
learned_labels = np.array(['normal','incivilidad','odio'])

In [60]:
learned_labels

array(['normal', 'incivilidad', 'odio'], dtype='<U11')

In [62]:
def auc_score(test_set, predicted_set):
    high_predicted = np.array([prediction[2] for prediction in predicted_set])
    medium_predicted = np.array(
        [prediction[1] for prediction in predicted_set]
    )
    low_predicted = np.array([prediction[0] for prediction in predicted_set])
    inc_test = np.where(test_set == "incivilidad", 1.0, 0.0)
    odio_test = np.where(test_set == "odio", 1.0, 0.0)
    normal_test = np.where(test_set == "normal", 1.0, 0.0)
    auc_high = roc_auc_score(inc_test, high_predicted)
    auc_med = roc_auc_score(odio_test, medium_predicted)
    auc_low = roc_auc_score(normal_test, low_predicted)
    auc_w = (
        normal_test.sum() * auc_low
        + odio_test.sum() * auc_med
        + inc_test.sum() * auc_high
    ) / (normal_test.sum() + odio_test.sum() + inc_test.sum())
    return auc_w


def evaluate(predicted_probabilities, y_test, labels):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador.
    # (que comunmente, es distinto a ['normal', 'odio', 'incivilidad'])
    predicted_labels = [
        labels[np.argmax(item)] for item in predicted_probabilities
    ]

    print("Matriz de confusión")
    print(
        confusion_matrix(
            y_test, predicted_labels, labels=["normal", "odio", "incivilidad"]
        )
    )

    print("\nReporte de clasificación:\n")
    print(
        classification_report(
            y_test, predicted_labels, labels=["normal", "odio", "incivilidad"]
        )
    )
    # Reorder predicted probabilities array.
    labels = labels.tolist()

    predicted_probabilities = predicted_probabilities[
        :,
        [
            labels.index("normal"),
            labels.index("odio"),
            labels.index("incivilidad"),
        ],
    ]

    auc = round(auc_score(y_test, predicted_probabilities), 3)
    print("Métricas:\n\nAUC: ", auc, end="\t")
    kappa = round(cohen_kappa_score(y_test, predicted_labels), 3)
    print("Kappa:", kappa, end="\t")
    accuracy = round(accuracy_score(y_test, predicted_labels), 3)
    print("Accuracy:", accuracy)
    print("------------------------------------------------------\n")
    return np.array([auc, kappa, accuracy])


In [68]:
predicted_probabilities

array([[0.04451904, 0.93562138, 0.01985958],
       [0.92305464, 0.01400349, 0.06294189],
       [0.01748722, 0.97010887, 0.01240395],
       ...,
       [0.08106995, 0.42701054, 0.49191946],
       [0.00552912, 0.98276937, 0.0117015 ],
       [0.00507274, 0.97307903, 0.02184826]])

In [74]:
y_test = np.array(list(map(lambda x: id2label[x],dataset['test']['label'])))

In [77]:
scores = evaluate(predicted_probabilities, y_test, learned_labels)

Matriz de confusión
[[1068   84  168]
 [ 116  537  111]
 [ 125   65 1391]]

Reporte de clasificación:

              precision    recall  f1-score   support

      normal       0.82      0.81      0.81      1320
        odio       0.78      0.70      0.74       764
 incivilidad       0.83      0.88      0.86      1581

    accuracy                           0.82      3665
   macro avg       0.81      0.80      0.80      3665
weighted avg       0.82      0.82      0.82      3665

Métricas:

AUC:  0.944	Kappa: 0.713	Accuracy: 0.817
------------------------------------------------------

